In [49]:
from math import degrees
from math import sqrt
from math import cos
from math import sin
from math import radians
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

vertical_length = 77
orizontal_length = 10

def create_data_short():
    
    def headingFromCoordinates(lat1, lon1, lat2, lon2): 
        if(lon2-lon1 < 0): #2nd and 3rd quadrant 
            heading =180 - degrees(np.arcsin((lat2-lat1)/(sqrt((lat2-lat1)**2 + (lon2-lon1)**2)))) 
        elif(lat2-lat1 < 0): #4th quadrant
            heading = 360 + degrees(np.arcsin((lat2-lat1)/(sqrt((lat2-lat1)**2 + (lon2-lon1)**2))))
        else:
            heading = degrees(np.arcsin((lat2-lat1)/(sqrt((lat2-lat1)**2 + (lon2-lon1)**2)))) #1st quadrant            
        return heading
    df = pd.read_csv('Data/Raw_boat_data')
    df = df[210:6830]
    
    spd = []
    dgr = []
    
    for i in range(5):
        for i in range(vertical_length):
            spd.append(np.random.normal(0.5, 0.3))
            dgr.append(np.random.normal(107.5, 5))
        for i in range(orizontal_length):
            spd.append(np.random.normal(0, 0.3))
            dgr.append(np.random.normal(7.5, 5))
        for i in range(vertical_length):
            spd.append(np.random.normal(-0.5, 0.3))
            dgr.append(np.random.normal(287.5, 5))
        for i in range(orizontal_length):
            spd.append(np.random.normal(0, 0.3))
            dgr.append(np.random.normal(7.5, 5))
    
    
    
    for i in range(vertical_length):
        spd.append(np.random.normal(0.5, 0.3))
        dgr.append(np.random.normal(107.5, 5))
    for i in range(orizontal_length):
        spd.append(np.random.normal(0, 0.3))
        dgr.append(np.random.normal(7.5, 5))
    for i in range(vertical_length):
        spd.append(np.random.normal(-0.5, 0.3))
        dgr.append(np.random.normal(287.5, 5))
    
    for i in range(len(dgr)):
        dgr[i] = dgr[i] + 140
        if dgr[i] > 360:
            dgr[i] = dgr[i] - 360    
    
    
    m = np.mean(df['acceleration'])
    s = np.std(df['acceleration'])
    
    acc = np.random.normal(m, s-0.2, len(spd))
    
    m0c = np.random.normal(0, 0.5, len(spd))
    # for i in range(5):
    #     m0c[47+(i*112)] = 10
    #     m0c[112+(i*112)] = -10

    m1c = np.random.normal(0, 0.5, len(spd))
    # for i in range(5):
    #     m1c[47+(i*112)] = -10
    #     m1c[112+(i*112)] = 10
        
    lats = [45.3596665575, 45.36225589, 45.36226747, 45.35967972, 45.35969351, 45.36227073,
            45.36228557, 45.35970006, 45.35971575, 45.36230304, 45.36230583,
            45.35971167, 45.35972416, 45.36232427, 45.36232928, 45.35974926,
            45.359755  , 45.36234141, 45.3623531 , 45.35975891, 45.35976923,
            45.36236122, 45.36236823, 45.3597792384]
    
    lons = [11.020597651, 11.01979384, 11.01985461, 11.02066511, 11.02074007, 11.01991931,
            11.0199881 , 11.02079678, 11.02087983, 11.02006662, 11.02012215,
            11.02093044 , 11.02100831, 11.02020208, 11.02025402, 11.02108572,
            11.02115422, 11.02033527, 11.02038858, 11.02118961, 11.02124338,
            11.02045991, 11.02053796, 11.0213068174]
    
    
    variance_lat = 0.000005
    variance_lon = 0.000002
    
    
    def noise_lon(coord):
        noise = coord - np.random.normal(coord, variance_lon)
        return coord + noise
    
    
    def noise_lat(coord):
        noise = coord - np.random.normal(coord, variance_lat)
        return coord + noise
    
    
    for i, elem in enumerate(lats):
        lats[i] = noise_lat(elem)   
    
    for j,elem in enumerate(lons):
        lons[j] = noise_lon(elem)
    
    
    g_lat = []
    
    g_lon = []
    
    for i in range(len(lats) - 1):
        start_lat = lats[i]
        start_lon = lons[i]
        g_lat.append(start_lat)
        g_lon.append(start_lon)
        
        l = sqrt((lons[i + 1] - lons[i]) ** 2 + (lats[i + 1] - lats[i]) ** 2)
        
        if i % 4 == 0:
            dl = l / vertical_length
            alpha = 180 - headingFromCoordinates(lats[i], lons[i], lats[i + 1], lons[i + 1])
            for j in range(1, vertical_length):
                g_lon.append(noise_lon((start_lon - (j * dl * cos(radians(alpha))))))
                g_lat.append(noise_lat(start_lat + (j * dl * sin(radians(alpha)))))
        if i % 4 == 1:
            dl = l / orizontal_length
            alpha = headingFromCoordinates(lats[i], lons[i], lats[i + 1], lons[i + 1])
            for j in range(1, orizontal_length):
                g_lon.append(noise_lon(start_lon + (j * dl * cos(radians(alpha)))))
                g_lat.append(noise_lat(start_lat + (j * dl * sin(radians(alpha)))))
        if i % 4 == 2:
            dl = l / vertical_length
            alpha = headingFromCoordinates(lats[i], lons[i], lats[i + 1], lons[i + 1])
            for j in range(1, vertical_length):
                g_lon.append(noise_lon(start_lon + (j * dl * cos(radians(alpha)))))
                g_lat.append(noise_lat(start_lat + (j * dl * sin(radians(alpha)))))
        if i % 4 == 3:
            dl = l / orizontal_length
            alpha = headingFromCoordinates(lats[i], lons[i], lats[i + 1], lons[i + 1])
            for j in range(1, orizontal_length):
                g_lon.append(noise_lon(start_lon + (j * dl * cos(radians(alpha)))))
                g_lat.append(noise_lat(start_lat + (j * dl * sin(radians(alpha)))))
    
    sins = []
    cosins = []
    for i, degree in enumerate(dgr):
        sins.append(np.sin(np.deg2rad(degree)))
        cosins.append(np.cos(np.deg2rad(degree)))
        
        
    columns = ["Speed", "Sin", "Cosin", 
               "Acceleration", "M0C", 
               "M1C", "Lat", "Lon"]
    
    # print(len(sins), len(cosins))
    # plt.plot(sins)
    # plt.plot(cosins)
    # plt.show()
    # 
    # print(len(dgr))
    # plt.plot(dgr)
    # plt.show()
    
    df = pd.DataFrame(columns=columns)
    df["Speed"] = spd
    df["Sin"] = sins
    df["Cosin"] = cosins
    df["Acceleration"] = acc
    df["M0C"] = m0c
    df["M1C"] = m1c
    df["Lat"] = g_lat
    df["Lon"] = g_lon
    
    df = df.drop(df.index[-10:])
    
 
    return df


In [64]:
df = create_data_short()
for i in range(300):
    data = create_data_short()
    # choice = np.random.choice(3, 1)
    # if choice == 0:
    #     data[331:] = np.zeros((663-331,7))
    # elif choice == 1:
    #     data[:332] = np.zeros((332,7))
    df = df.append(data)
    
    
print(df)

         Speed       Sin     Cosin  Acceleration       M0C       M1C  \
0     1.133309 -0.936406 -0.350917     -0.485694  0.510700 -0.988781   
1     0.472815 -0.970093 -0.242732     -0.225197 -0.392356 -0.251745   
2     0.106405 -0.857418 -0.514620     -0.020861  0.285421  0.647040   
3     0.868479 -0.883803 -0.467859     -0.916502 -0.057443 -0.470730   
4     0.780787 -0.875559 -0.483111     -0.334817 -0.347971 -0.848157   
...        ...       ...       ...           ...       ...       ...   
1019 -0.822567  0.946247  0.323444     -1.562378 -0.030465  0.036135   
1020 -0.473991  0.897345  0.441329     -1.252587 -0.274814 -0.218052   
1021 -0.699133  0.914698  0.404137     -1.563437  0.330764 -0.037555   
1022 -0.764374  0.878833  0.477129      1.621202  0.341229 -0.562694   
1023 -0.477115  0.955964  0.293483      0.267929 -0.198414  0.100223   

            Lat        Lon  
0     45.359665  11.020597  
1     45.359697  11.020587  
2     45.359730  11.020577  
3     45.359762  11

In [65]:

df = df.to_csv("Data/Boat_nominal_data/Boat_nominal_sequences.csv")
